In [1]:
import datetime
import json
import warnings

import numpy as np
import rasterio
import requests
import stac
import wtss
import xarray as xr
from pyproj import CRS, Proj, transform
from rasterio.windows import Window

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from eocube import DataCube, config

warnings.filterwarnings("ignore")

config.ACCESS_TOKEN = "a6gB61oddyHxMAZ08y098ynEq0s1hQ3siDZao245CV"

In [2]:
eocube_service = DataCube()

In [10]:
# bbox = [-56.5009689, -10.7164420, -56.3526535, -10.8238641] # Mato Grosso
bbox = [-46.01348876953125, -23.08478515994374, -45.703125, -23.34856015148709] # Interior de SP São José

items = eocube_service.getItems(
    collections=["CB4_64_16D_STK-1"],
    bbox=bbox,
    start_date="2018-08-01",
    end_date="2019-07-31"
)

In [8]:
print(items[0]['assets']['BAND13']['href'])

https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/CB4_64_16D_STK/v001/022027/2019-07-28_2019-08-12/CB4_64_16D_STK_v001_022027_2019-07-28_2019-08-12_BAND13.tif?access_token=a6gB61oddyHxMAZ08y098ynEq0s1hQ3siDZao245CV


In [11]:
eocube_service.createCube()

True

In [ ]:
eocube_service.getCubeByBand('red')

In [13]:
band = 'red'
longitude = list(range(0, 500))
latitude = list(range(0, 500))

x_data = {}

for image in eocube_service.getCube():
    date = datetime.datetime.strptime(
        image.item["properties"]["datetime"],
        '%Y-%m-%dT%H:%M:%S'
    )
    data = image.getBand(band)
    
    x_data[date] = data
    
timeline = sorted(list(x_data.keys()))
data_timeline = []

for time in timeline:
    data_timeline.append(x_data[time])

data = xr.DataArray(
    np.array(data_timeline),
    coords=[timeline, longitude, latitude],
    dims=["time", "laitude", "latitude"]
)

data.attrs = eocube_service.getDescription("CB4_64_16D_STK-1")

In [14]:
data

<xarray.DataArray (time: 24, laitude: 500, latitude: 500)>
array([[[1689, 2163, 2381, ...,  897,  869,  864],
        [2021, 1575, 1142, ...,  892,  856,  858],
        [2595, 1462, 1732, ...,  904,  875,  886],
        ...,
        [ 670,  633,  638, ..., 1076, 1129,  748],
        [ 807,  798,  823, ..., 1006, 1072, 1154],
        [ 579,  806,  788, ...,  936, 1014, 1179]],

       [[1795, 1475, 1042, ..., 1235, 1017, 1091],
        [1764, 1707, 1540, ..., 1286, 1036,  925],
        [1868, 1855, 1748, ..., 1178, 1010,  942],
        ...,
        [ 619,  773, 1182, ...,  802,  764,  760],
        [ 869, 1062, 1137, ...,  795,  763,  778],
        [ 911,  990, 1036, ...,  800,  767,  805]],

       [[1512, 1301,  872, ..., 1370, 1080, 1001],
        [1649, 1402, 1089, ..., 1261,  990,  993],
        [1777, 1775, 1647, ..., 1010,  972, 1028],
        ...,
...
        ...,
        [1122,  984,  694, ..., 1052,  864, 1325],
        [ 867,  774,  699, ..., 1233,  897, 1303],
        [1144, 1079, 1042, ..., 1405, 1171, 1010]],

       [[1743, 1613, 1368, ..., 1984, 1956, 1933],
        [1702, 2069, 1725, ..., 1956, 1945, 1932],
        [1838, 2347, 2290, ..., 1971, 1970, 1950],
        ...,
        [1695, 1547,  978, ...,  555,  541,  533],
        [1412, 1388,  882, ...,  564,  545,  520],
        [1271, 1237, 1150, ...,  557,  548,  526]],

       [[1557, 1401, 1053, ..., 1631, 1599, 1578],
        [1598, 1619, 1328, ..., 1612, 1604, 1565],
        [1525, 1729, 1510, ..., 1557, 1540, 1528],
        ...,
        [1326, 1119,  845, ...,  502,  495,  521],
        [1307, 1052,  867, ...,  528,  525,  543],
        [1279, 1077,  998, ...,  527,  549,  561]]], dtype=int16)
Coordinates:
  * time      (time) datetime64[ns] 2018-07-28 2018-08-13 ... 2019-07-28
  * laitude   (laitude) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
  * latitude  (latitude) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
Attributes:
    id:            CB4_64_16D_STK-1
    title:         CBERS-4 - AWFI - Cube Stack 16 days - v001
    descriptions:  [{'lang': 'en', 'description': "This datacube was generate...
    extent:        {'spatial': {'bbox': [[-79.267539, -35.241045, -29.421667,...
    properties:    {'eo:gsd': 64.0, 'eo:bands': [{'name': 'BAND13', 'common_n...

In [17]:
data.loc['2018-07-28':'2019-08-14', 50, 50]

<xarray.DataArray (time: 24)>
array([ 764,  801,  791,  743, 1179,  742,  703,  712,  872,  604,  631,
        975,  451, 1442,  932,  459,  556,  502,  354, 1096,  495,  544,
        617,  625], dtype=int16)
Coordinates:
  * time      (time) datetime64[ns] 2018-07-28 2018-08-13 ... 2019-07-28
    laitude   int64 50
    latitude  int64 50
Attributes:
    id:            CB4_64_16D_STK-1
    title:         CBERS-4 - AWFI - Cube Stack 16 days - v001
    descriptions:  [{'lang': 'en', 'description': "This datacube was generate...
    extent:        {'spatial': {'bbox': [[-79.267539, -35.241045, -29.421667,...
    properties:    {'eo:gsd': 64.0, 'eo:bands': [{'name': 'BAND13', 'common_n...

In [20]:
link = items[0]['assets']['BAND13']['href']

import rasterio
from pyproj import CRS, Proj, transform

with rasterio.open(link) as dataset:
    coord = dataset.transform * (0, 2)
    
    lon, lat = transform(
        dataset.crs.wkt,
        Proj(init=CRS.from_string("EPSG:4326")),
        coord[0], coord[1]
    )
print(lon, lat)

-47.83096256503787 -22.874661630385692


In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(raster.getRGB())
plt.tight_layout()

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
colormap = plt.get_cmap('Reds', 1000)
plt.imshow(
    raster.getBand("red"),
    cmap=colormap
)
plt.tight_layout()
plt.colorbar()

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
colormap = plt.get_cmap('Greens', 1000)
plt.imshow(
    raster.getNDVI(),
    cmap=colormap
)
plt.tight_layout()
plt.colorbar()

plt.show()

In [ ]:
plt.title("NDVI Histogram")
plt.hist(raster.getNDVI())
plt.show()